Data preparation for modeling

## Environment Set-Up

### Load relevant Python Packages

In [1]:
# Importing the most important modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib import pyplot

# For data reading and data export
import os, glob
from pathlib import Path
from pandas import read_csv

# For data cleaning
from datetime import datetime

# For data exploration

# For data visualisation - WordCloud
from wordcloud import WordCloud

## Loading Datasets & Preparation for Modeling

In [2]:
#data has been saved using a .pkl file. 
path = './data/df_ride.pkl'
df = pd.read_pickle(path)
df.head(2)

,When,Type,Gear,Name,Dist km,Elv m,Elapsed Time,Moving Time,Start Time,Day of Week,...,Power 250-300W,Power 300-350W,Power 350-400W,Power 400-450W,Power 450W+,City,State,Device/App,Activity Id,filename
0,2021-03-28 14:15:27,Ride,Canyon Ultimate CF SLX,Fahrt am Nachmittag,90796.6,797.0,12497,12014,51327,Sunday,...,1318,742,307,140,127.0,NaN,NaN,Garmin Edge 1030,5026851534,95
1,2021-03-27 12:32:34,Ride,Canyon Ultimate CF SLX,#Südflügel - #Lockdownduo - Samstagscross,95537.9,769.0,18614,16282,45154,Saturday,...,424,249,150,89,88.0,NaN,NaN,Garmin Edge 1030,5020466545,95


In [3]:
# Never a bad idea - Quickly check the number of rows (observations) and columns (features) in the df
df.shape

(360985, 54)

In [4]:
# Let us check some basic information for the columns (for example the data types)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360985 entries, 0 to 416001
Data columns (total 54 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   When                  360985 non-null  object 
 1   Type                  360985 non-null  object 
 2   Gear                  344166 non-null  object 
 3   Name                  360981 non-null  object 
 4   Dist km               360985 non-null  float64
 5   Elv m                 360985 non-null  float64
 6   Elapsed Time          360985 non-null  int64  
 7   Moving Time           360985 non-null  int64  
 8   Start Time            360985 non-null  int64  
 9   Day of Week           360985 non-null  object 
 10  Speed km/h            360985 non-null  float64
 11  Max Speed km/h        360985 non-null  float64
 12  Pace /km              360985 non-null  float64
 13  Max Pace /km          360985 non-null  float64
 14  Pace /100m            360985 non-null  float64
 15  

_________

## Data Cleaning / Feature Engineering

The following is pretty much dthe same as in [CyPer_EDA](CyPer_EDA.ipynb) with just small but important differences for the following predictive modeling. 

In [5]:
#To work with the `When` column and the data in it we have to transform the data type. 
#We do this with the function: `to_datetype`. By this function we convert the date type into *datetime*.
df.When = pd.to_datetime(df.When)

In [6]:
# Convert timestamp ('When') to datetime object
# Extract additional features from timestamp column
df = df.assign(
    timestamp = lambda x: pd.to_datetime(x['When']), 
    date = lambda x: x['When'].dt.date,
    year = lambda x: x['When'].dt.year,
    month = lambda x: x['When'].dt.month,
    day = lambda x: x['When'].dt.day,
    dayofyear = lambda x: x['When'].dt.dayofyear,
    hour = lambda x: x['When'].dt.hour,
    minute = lambda x: x['When'].dt.minute,
    second = lambda x: x['When'].dt.second,
)

In [7]:
# Rename columns in a more pythonic way
df = df.rename(columns={
    'Type': 'type', 
    'Gear': 'gear', 
    'Name': 'name',
    'Dist km': 'dist_km',
    'Elv m': 'elv_m',
    'Elapsed Time': 'elapsed_time',
    'Moving Time': 'moving_time',
    'Start Time': 'start_time',
    'Day of Week': 'day_of_week',
    'Speed km/h': 'speed_km/h',
    'Max Speed km/h': 'max_speed_km/h',
    'Pace /km': 'pace_min/km', 
    'Max Pace /km': 'max_pace_min/km', 
    'Pace /100m': 'pace_min/100m',
    'Max Pace /100m': 'max_pace_min/100m',
    'Pwr W': 'power_W',
    'Weighted Avg Pwr W': 'weighted_avg_power_W',
    'Max Pwr W': 'max_power_W',
    'Cad': 'cad',
    'Heart': 'heart',
    'Max Heart': 'max_heart',
    'Elv High m': 'elv_high_m',
    'Elv Low m': 'elv_low_m',
    'Efficiency %': 'efficiency_%',
    'Elev/Dist m/km': 'elv/dist_m/km',
    'Elev/Time m/h': 'elv/time_m/h',
    'W/HR': 'w/hr',
    'Speed/HR': 'speed/hr',
    'Temp °C': 'temp_°C',
    'Cal': 'cal',
    'Energy kJ': 'energy_kJ',
    'Dist start to end km': 'dist_start_to_end_km',
    'H/R Zone 1': 'h/r_zone1',
    'H/R Zone 2': 'h/r_zone2',
    'H/R Zone 3': 'h/r_zone3',
    'H/R Zone 4': 'h/r_zone4',
    'H/R Zone 5': 'h/r_zone5',
    'Power 0W': 'power_0W',
    'Power 0-50W': 'power_0-50W',
    'Power 50-100W': 'power_50-100W',
    'Power 100-150W': 'power_100-150W',
    'Power 150-200W': 'power_150-200W',
    'Power 200-250W': 'power_200-250W',
    'Power 250-300W': 'power_250-300W',
    'Power 300-350W': 'power_300-350W',
    'Power 350-400W': 'power_350-400W',
    'Power 400-450W': 'power_400-450W',
    'Power 450W+': 'power_450W+',
    'Description': 'description',
    'City': 'city',
    'State': 'state',
    'Device/App': 'device/app',
    'Activity Id': 'activity_id',
    'filename': 'cyclist_id'
    }
)

In [8]:
# Convert and transform data
df['dist_km'] = (df['dist_km'] / 1000).round(2)
df['speed_km/h'] = (df['speed_km/h'] * 3.6).round(1)
df['max_speed_km/h'] = (df['max_speed_km/h'] * 3.6).round(1)
df['power_W'] = df['power_W'].round(0)
df['heart'] = df['heart'].round(0)
df['temp_°C'] = df['temp_°C'].round(0)
df['efficiency_%'] = df['efficiency_%'].round(2)
df['dist_start_to_end_km'] = (df['dist_start_to_end_km'] / 1000).round(2)

In [9]:
# drop duplicates
df.drop_duplicates(subset="activity_id", inplace=True)

In [10]:
# dist_km has 2902 (1.0%) zeros
# drop all rows with a 'dist_km' of 0
df = df[df.dist_km != 0]

In [11]:
# 'speed_km/h' has 2937 (1.1%) zeros
# drop all rows with a 'speed_km/h' of 0
df = df[df['speed_km/h'] != 0]

In [12]:
# delate all activities with an average speed > 70km/h
df = df[df['speed_km/h'] <= 70]

In [13]:
# delet all rows that have 'elv_m' is 0 and the activitie is before 01.02.2018
df = df.drop(df[(df['elv_m'] == 0) & (df['When'] <= '2018-02-01')].index)

In [14]:
# delate all 'temp_°C' with an temp greater then 70°C
df = df[df['temp_°C'] <= 70]

In [15]:
# replace all NaN in'temp_°C' woth median of 'temp_°C'
df['temp_°C'] = df['temp_°C'].fillna((df['temp_°C'].median()))

In [16]:
df = df[df['heart'].notna()]

In [17]:
df['heart'] = df['heart'].mask(df['heart'] == 0, df['heart'].median(skipna=True))

In [18]:
# replace all 0 in'energy_kJ' with mean of 'energy_kJ'
df['energy_kJ'] = df['energy_kJ'].mask(df['energy_kJ'] == 0, df['energy_kJ'].mean(skipna=True))

In [19]:
# replace all 0 in'cal' with mean of 'cal'
df['cal'] = df['cal'].mask(df['cal'] == 0, df['cal'].mean(skipna=True))

In [20]:
# replace all 0 in'cad' with median of 'cad'
df['cad'] = df['cad'].mask(df['cad'] == 0, df['cad'].median(skipna=True))

In [21]:
# delete all activities with dist_km < 2km
df = df.drop(df[(df['dist_km'] <= 1.99)].index)

In [22]:
# replace all NaN in'elv_high_m' with mean of 'elv_high_m'
df['elv_high_m'] = df['elv_high_m'].fillna((df['elv_high_m'].mean()))

In [23]:
# replace all NaN in'elv_low_m' with mean of 'elv_low_m'
df['elv_low_m'] = df['elv_low_m'].fillna((df['elv_low_m'].mean()))

In [24]:
# exclute all rows without input for power_... and h/r_zone...
df = df[df[['h/r_zone1', 'h/r_zone2', 'h/r_zone3', 'h/r_zone4', 'h/r_zone5']].ne(0, axis=0).any(axis=1)]

In [25]:
#Drop columns with to many missing values
#max_power_W has 194319 (69.6%) missing values
#weighted_avg_power_W has 193704 (69.4%) zeros
df.drop(['When', 'type', 'gear', 'city', 'state', 'device/app', 'activity_id', 'cyclist_id', 'power_0W', 'power_0-50W', 'power_50-100W', 'power_100-150W',
       'power_150-200W', 'power_200-250W', 'power_250-300W', 'power_300-350W','max_pace_min/km','max_pace_min/100m',
       'power_350-400W', 'power_400-450W','date', 'timestamp', 'power_450W+', 'max_power_W', 'weighted_avg_power_W', 'name', 'dist_start_to_end_km', 'max_speed_km/h', 'w/hr'], axis=1, inplace=True)

In [26]:
df.reset_index(drop=True, inplace=True)

In [27]:
# Checking for NaN values in the df_model data frame
df.isna().values.any()

False

In [28]:
# Find out the lines with inf values
df[(df == np.inf).any(axis=1)]

,dist_km,elv_m,elapsed_time,moving_time,start_time,day_of_week,speed_km/h,pace_min/km,pace_min/100m,power_W,...,h/r_zone3,h/r_zone4,h/r_zone5,year,month,day,dayofyear,hour,minute,second
83530,17.33,48.0,19370,3454,30726,Wednesday,18.1,199.4,19.94,72.0,...,0,0,0,2020,7,1,183,8,32,6
84042,17.91,34.0,49733,3302,36809,Saturday,19.5,184.4,18.44,23.0,...,0,0,0,2017,10,21,294,10,13,29


In [29]:
# delet the lines with inf values
df = df.drop(df.index[[83530,84042]])

In [30]:
# delete all rows with 0 in'power_W'
df = df[df['power_W'] != 0]

### Creat Dummy Varaible

The `"day_of_week"` column is categorical, not numeric. So we have to convert that to a one-hot/dummy:

In [31]:
#create dummie-variables
day_of_week_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week', drop_first=True)

In [32]:
df = pd.concat([df.drop(['day_of_week'], axis = 1), day_of_week_dummies], axis = 1)

## Export of Data

The data modification ends at this point. The DataFrame (df) is organized, cleaned and modified. For purity and performance reasons, the Machine Learning part of this project will be continued in another Jupyter Notebook. The interface between the Jupyter Notebooks will be given using a `.pkl`file.
- `df_model.pkl` --> DataFrame with only the most important features ready for computational demanding ML models

In [33]:
df_model = df.copy()

In [34]:
# save the cleaned df as .pkl file
path = './data/df_model.pkl'
df_model.to_pickle(path)
Path(path).is_file()

print('This cell was last run on: ')
print(datetime.now())

This cell was last run on: 
2021-04-25 15:03:18.975093
